First we import praw and configure our reddit API to be able to retrieve submissions

In [2]:
import praw
from psaw import PushshiftAPI
import json

reddit = praw.Reddit(client_id='D52pMTitN_4Okg',
                     client_secret='6sOjBsPe-EryAiLdvgoRLyFKrVA',
                     user_agent='ChomskyBot',
                     username='ChomskyBot',
                     password='CaRaGio12625')

In [3]:
subreddit = reddit.subreddit('amitheasshole')
api = PushshiftAPI(reddit)

This method will label AITA posts according to their comments or flair

In [4]:
import nltk
from nltk.tokenize import word_tokenize
def label(p):
    labels = ['YTA', 'NTA', 'ESH', 'NAH']
    label_score={label:0 for label in labels}
    
    #If the post hasn't been "flaired" (assigned a label by mods) then I calculate it myself
    if p['link_flair_text'] == None:
        p['label'] = comment_label(p, labels, label_score)
        
    #If the post has been flaired with one of the labels I assign it a 100% probability, otherwise
    #for posts with tags like META I give the post no label
    else:
        if p['link_flair_text'].lower() == 'asshole':
            label_score['YTA'] = 1
        elif p['link_flair_text'].lower() == 'not the a-hole':
            label_score['NTA'] = 1
        elif p['link_flair_text'].lower() == 'everyone sucks':
            label_score['ESH'] = 1
        elif p['link_flair_text'].lower() == 'no a-holes here':
            label_score['NAH'] = 1
        else:
            label_score = None
    
    p['label'] = label_score

    
def comment_label(p, labels, label_score):
    
    score = 0
    label = False
    
    #I loop through the comments and try to look for one of the keywords, if I find it I add its score to the
    #tags score
    for c in p['comments']:
        words = word_tokenize(c['body'])
        for w in words:
            w = w.upper()
            if w in labels:
                label = True
                label_score[w] += c['score']
                score += c['score']
    
    for k, v in label_score.items():
        if score != 0:
            label_score[k] = v/score
            
    
    #If no comment contained any of the keywords for a label then I give the post no label
    if not label:
        label_score = None
        
    return label_score
            


This method will retrieve all posts from a specific subreddit from a specific date onwards. After 500 posts are retrieved and labelled they will be stored in a given file


In [9]:
import datetime as dt
def post_retrieval(file, subreddit, date):
    
    submissions = api.search_submissions(after=date, subreddit=subreddit, limit=None, sort="date:asc")
    
    for submission in submissions:
        #Retrieve set variables from the submission object and store them in a post dictionary
        post = vars(submission)
        post_dict = {field:post[field] for field in post_fields[:-2]}
        
        #Retrieve 10 top comments and stors them as a dict, this is used for labelling of non-flaired posts
        comments = []
        if post_dict['link_flair_text'] == None:
            for i in range(1, min(11, len(submission.comments.list()))):
                comment = submission.comments.list()[i]
                content = vars(comment)
                comment_dict = {field:content[field] for field in comment_fields if field in content}
                if 'body' in content:
                    comments.append(comment_dict)
        
        post_dict['comments'] = comments
        label(post_dict)
        posts.append(post_dict)
        
        #After 50 posts are added they're written into the data file, this is done to ensure that the progress
        #is not lost in case of internet connection issues or other problems
        if len(posts)%500 == 0:
            print("We're at: " + str(len(posts)))
            with open(file, 'w+') as f:
                json.dump(posts, f)
                

In [ ]:
Sets the fields to be retrieved and the dates from which we start retrieving

In [ ]:

posts = []
post_dict = {}

#Fields to be retrieved from submissions and comments and subreddit from which we will be extracting them
comment_fields = ('id', 'body', 'link_id', 'permalink', 'score', 'subreddit_id')
post_fields = ('id','name', 'created_utc', 'title', 'link_flair_text', 'selftext', 'score', 'comments', 'label')
subreddit = 'amitheasshole'

#Period from which we want to retrieve the comments
#date = int(dt.datetime(2019,1,1).timestamp())
date = int(1548904042.0)

post_retrieval("data2.json", subreddit, date) 

We're at: 500
We're at: 1000
We're at: 1500
We're at: 2000
We're at: 2500
We're at: 3000
We're at: 3500
We're at: 4000
We're at: 4500
We're at: 5000
We're at: 5500
We're at: 6000
We're at: 6500
We're at: 7000
We're at: 7500
We're at: 8000
We're at: 8500
We're at: 9000
We're at: 9500
We're at: 10000
We're at: 10500
We're at: 11000
We're at: 11500
We're at: 12000
We're at: 12500
We're at: 13000
We're at: 13500
We're at: 14000
We're at: 14500
We're at: 15000
We're at: 15500
We're at: 16000
We're at: 16500
We're at: 17000
We're at: 17500
